In [28]:
%load_ext Cython

import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import two_qubit_toolkit.tool as kit
from tqdm import tqdm_notebook as tqdm

# 記録用
from datetime import datetime
import os


def task_gen(m):
    # Clifford Sequence Generator.
    Gen = []
    for i in range(0, m):
        Gen.append(i)

    ye = np.random.choice(Gen, m, replace = False)
    rere = ye

    return rere

In [4]:
os.chdir('C:\\Users\\sho60\\Google_drive\\UTSB\\QCalc\\Q_simu\\src\\gates\\group_table2')
cli2 = np.load('cliffords_2Qnp.npy')
conjs = np.conjugate(file_2q)
test_2q = np.load('pregt_twoq5000.npy')

## hashテーブルを作る
### 方針
- 位相の自由度をキャンセルする。ゼロじゃない要素の共役で割る。
- ゼロの要素数（位置）で場合分け非ゼロ要素を2進数に変換。
- 2進数を単純に結合し、新しく10進整数に直す。
- それをストリングに直し辞書のkeyとする。

In [86]:
%%cython
import numpy as np
cimport numpy as np

def count(np.ndarray[np.complex128_t, ndim=3] cli2):
    cdef int i, j, k, l, m, n, o, p, q, r
    cdef int cmin = 0
    cdef int cmax = 11520
    i = 0
    j = 0
    k = 0
    l = 0
    n = 0
    o = 0
    p = 0
    q = 0
    for m in range(cmin, cmax):
        if cli2[m, 0, 0] != 0 and cli2[m, 1, 0] == 0:
            i += 1
        elif cli2[m, 1, 0] != 0 and cli2[m, 2, 0] == 0:
            j += 1
        elif cli2[m, 2, 0] != 0 and cli2[m, 3, 0] == 0:
            k += 1
        elif cli2[m, 3, 0] != 0 and cli2[m, 0, 0] == 0:
            l += 1
        elif cli2[m, 0, 0] != 0:
            n += 1
        elif cli2[m, 1, 0] != 0:
            o += 1
        elif cli2[m, 2, 0] != 0:
            p += 1
        elif cli2[m, 3, 0] != 0:
            q += 1
    return [i, j, k, l, n, o, p, q]
        

In [87]:
c = count(cli2)
c

[1728, 1728, 960, 960, 6144, 0, 0, 0]

In [88]:
sum(c)

11520

## Clifford群の特徴
- 第一列の要素をカウントしたところ、ちょうど11520で、かぶりがなかった

### 以下でバイナリに変換する方法を考える

In [122]:
a = bin(int(-cli2[0, 0, 0].real * 1000)) #+ bin(int(cli2[2, 0, 0].real * 1000))
print(a)
print(type(a))
int(a, 0)

-0b1111101000
<class 'str'>


-1000

In [111]:
wa = '' + 'a'
dic = {}
dic['{}'.format(wa)] = '3'
dic

{'a': '3'}

In [55]:
cli2[10] #/ cli2[10, 0, 0]

array([[ 0.5-0.5j, -0.5+0.5j,  0. +0.j ,  0. +0.j ],
       [ 0.5+0.5j,  0.5+0.5j,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0.5-0.5j, -0.5+0.5j],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.5j,  0.5+0.5j]])

In [134]:
%%cython
import numpy as np
cimport numpy as np

def hash_table_gen(np.ndarray[np.complex128_t, ndim=3] cli2):
    cdef int m, i, j, k, l, n, o, p, q
    cdef int tan = 3
    cdef int saku = 3
    cdef int cmin = 0
    cdef int cmax = 11520
    cdef int appr = 1000
    cdef np.ndarray keygen
    cdef str a, b, c, d, e, key
    cdef dict table
    
    k = 0
    l = 0
    n = 0
    o = 0
    p = 0
    q = 0
    
    table = {}
    for m in range(cmin, cmax):
        key = ''
        if cli2[m, 0, 0] != 0 and cli2[m, 1, 0] == 0:
            k += 1
            key = '1'
            keygen = cli2[m] / cli2[m, 0, 0]
            for i in range(cmin , tan):
                for j in range(cmin, saku):
                    key = key + bin(int(keygen[i, j].real * appr)) + 'i' + bin(int(keygen[i, j].imag * appr))
            table[key] = int(m)
            
        elif cli2[m, 1, 0] != 0 and cli2[m, 2, 0] == 0:
            l += 1
            key = '2'
            keygen = cli2[m] / cli2[m, 1, 0]
            for i in range(cmin , tan):
                for j in range(cmin, saku):
                    key = key + bin(int(keygen[i, j].real * appr)) + 'i' + bin(int(keygen[i, j].imag * appr))
            table[key] = int(m)
            
        elif cli2[m, 2, 0] != 0 and cli2[m, 3, 0] == 0:
            n += 1
            key = '3'
            keygen = cli2[m] / cli2[m, 2, 0]
            for i in range(cmin , tan):
                for j in range(cmin, saku):
                    key = key + bin(int(keygen[i, j].real * appr)) + 'i' + bin(int(keygen[i, j].imag * appr))
            table[key] = int(m)
            
        elif cli2[m, 3, 0] != 0 and cli2[m, 0, 0] == 0:
            o += 1
            key = '4'
            keygen = cli2[m] / cli2[m, 3, 0]
            for i in range(cmin , tan):
                for j in range(cmin, saku):
                    key = key + bin(int(keygen[i, j].real * appr)) + 'i' + bin(int(keygen[i, j].imag * appr))
            table[key] = int(m)
            
        else:
            p += 1
            key = '5'
            keygen = cli2[m] / cli2[m, 0, 0]
            for i in range(cmin , tan):
                for j in range(cmin, saku):
                    key = key + bin(int(keygen[i, j].real * appr)) + 'i' + bin(int(keygen[i, j].imag * appr))
            table[key] = int(m)
            
    return [table, [k, l, n, o, p]]
    
    

In [135]:
%time tableq = hash_table_gen(cli2)

Wall time: 226 ms


In [136]:
len(tableq[0])

9781

In [137]:
sum(tableq[1])

11520

In [138]:
tableq[0]

{'10b1111101000i0b00b0i0b00b0i0b00b0i0b00b1111101000i0b00b0i0b00b0i0b00b0i0b00b1111101000i0b0': 722,
 '20b0i0b00b1111101000i0b00b0i0b00b1111101000i0b00b0i0b00b0i0b00b0i0b00b0i0b00b0i0b0': 409,
 '20b0i0b0-0b1111101000i0b00b0i0b00b1111101000i0b00b0i0b00b0i0b00b0i0b00b0i0b00b0i0b0': 410,
 '10b1111101000i0b00b0i0b00b0i0b00b0i0b0-0b1111101000i0b00b0i0b00b0i0b00b0i0b00b1111101000i0b0': 731,
 '2-0b1111101000i0b00b0i-0b11111001110b0i0b00b1111101000i0b00b0i-0b11111010000b0i0b00b0i0b00b0i0b0-0b1111101000i0b0': 1432,
 '20b1111101000i0b00b0i0b11111001110b0i0b00b1111101000i0b00b0i-0b11111010000b0i0b00b0i0b00b0i0b00b1111101000i0b0': 1423,
 '2-0b1111101000i0b00b0i0b11111001110b0i0b00b1111101000i0b00b0i0b11111010000b0i0b00b0i0b00b0i0b0-0b1111101000i0b0': 6,
 '20b1111101000i0b00b0i-0b11111001110b0i0b00b1111101000i0b00b0i0b11111010000b0i0b00b0i0b00b0i0b00b1111101000i0b0': 7,
 '20b0i-0b11111010000b0i-0b11111001110b0i0b00b1111101000i0b0-0b1111101000i0b00b0i0b00b0i0b00b0i0b00b0i-0b1111101000': 8,
 '20b0i0b